In [1]:
import pandas as pd
from pandas import concat
import statsmodels.api as sm

from sklearn import metrics
from sklearn import preprocessing as prep
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels as sms
import statsmodels.tsa as tsa
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pmdarima as pm
import pandas_datareader as web
from arch import  arch_model
from math import sqrt
import numpy as np


In [2]:
from sklearn.utils import check_array
def MAPE(Y_test, preds): 
    Y_test, preds = np.array(Y_test), np.array(preds)
    return np.mean(np.abs((Y_test - preds) / Y_test)) * 100

In [3]:
stock = web.get_data_yahoo("T",
start = "2021-01-01",
end = "2021-11-17")
y=stock['Adj Close']

Y_t=y.T
Y_t=Y_t.iloc[1:167]
Y_train=Y_t

model = pm.auto_arima(Y_train, start_p=3, start_q=3,start_d=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=6, max_q=6, max_d=2, # maximum p and q
                      m=1,              # frequency of series
                      d=1,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      #start_P=0, 
                      #D=1, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

print(model.summary())

#ARIMA model
Errors_000=[]
for i in range(0,13):
    Y_train=Y_t.iloc[:153+i]
    Y_test=Y_t.iloc[153+i:154+i]
    
    arima_model = ARIMA(Y_train, order=model.order)
    arima_model_fit = arima_model.fit()
    preds_arima, se, conf = arima_model_fit.forecast(len(Y_test),alpha=0.3)  
    preds = pd.Series(preds_arima, index=Y_test.index)
    
    mse = mean_squared_error(Y_test, preds)
    rmse = sqrt(mean_squared_error(Y_test, preds))
    mae = mean_absolute_error(Y_test, preds)
    mape=MAPE(Y_test, preds)
    Error_i = ([mse, rmse, mae, mape])
    Errors_000.append(Error_i)
    Errors_A=pd.DataFrame(Errors_000)



#Garch model   
Errors_001=[]
for i in range(0,13): 
    Y_train=Y_t.iloc[:153+i]
    Y_test=Y_t.iloc[153+i:154+i]

    garch = arch_model(Y_train, vol="Garch", p=6, o=2, q=6, dist="skewt")
    garch_fitted = garch.fit()
    garch_forecast = garch_fitted.forecast(horizon=1)
    garch_pred_mean = garch_forecast.mean['h.1'].iloc[-1]
    preds = pd.DataFrame([garch_pred_mean])

    mse = mean_squared_error(Y_test, preds)
    rmse = sqrt(mean_squared_error(Y_test, preds))
    mae = mean_absolute_error(Y_test, preds)
    mape=MAPE(Y_test, preds)
    Error_i = ([mse, rmse, mae, mape])
    Errors_001.append(Error_i)
    Errors_B=pd.DataFrame(Errors_001)

    
#ARIMA-GARCH model  
Errors_002=[]
for i in range(0,13):
    Y_train=Y_t.iloc[:153+i]
    Y_test=Y_t.iloc[153+i:154+i]
    
    arima_model = ARIMA(Y_train, order=(model.order))
    arima_model_fit = arima_model.fit()
    arima_residuals = arima_model_fit.resid
    predicted_mu = arima_model_fit.forecast(len(Y_test))[0]

    garch = arch_model(arima_residuals, vol="Garch", p=7, o=2, q=7, dist="gaussian")
    garch_fitted = garch.fit()
    garch_forecast = garch_fitted.forecast(horizon=1)
    garch_pred_mean = garch_forecast.mean['h.1'].iloc[-1]
    garch_pred_var = garch_forecast.variance['h.1'].iloc[-1]


    # Combine both models' output: yt = mu + et
    preds = predicted_mu + garch_pred_var
    
    mse = mean_squared_error(Y_test, preds)
    rmse = sqrt(mean_squared_error(Y_test, preds))
    mae = mean_absolute_error(Y_test, preds)
    mape=MAPE(Y_test, preds)
    Error_i = ([mse, rmse, mae, mape])
    Errors_002.append(Error_i)
    Errors_C=pd.DataFrame(Errors_002)
    
    
# print(Errors_A[3].mean())
# print(Errors_B[3].mean())
# print(Errors_C[3].mean())

Performing stepwise search to minimize aic
 ARIMA(3,1,3)(0,0,0)[0] intercept   : AIC=83.216, Time=0.43 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=86.606, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=85.187, Time=0.03 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=83.308, Time=0.03 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=84.613, Time=0.01 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=85.216, Time=0.26 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=80.821, Time=0.33 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=79.432, Time=0.17 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=81.905, Time=0.11 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=81.079, Time=0.07 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=82.440, Time=0.10 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=84.643, Time=0.18 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=82.859, Time=0.10 sec
 ARIMA(2,1,2)(0,0,0)[0]             : AIC=77.440, Time=0.11 sec
 ARIMA(1,1,2)(0,0,0)[0]             : AIC=79.919, Time=0.04 s

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.AR

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.AR

Iteration:      1,   Func. Count:     20,   Neg. LLF: 904.6409011079679
Iteration:      2,   Func. Count:     43,   Neg. LLF: 15401.64598536754
Iteration:      3,   Func. Count:     63,   Neg. LLF: 9543.149619033882
Iteration:      4,   Func. Count:     83,   Neg. LLF: 7552.7844397323415
Iteration:      5,   Func. Count:    103,   Neg. LLF: 7693.006835331504
Iteration:      6,   Func. Count:    123,   Neg. LLF: 210.6866564621837
Iteration:      7,   Func. Count:    143,   Neg. LLF: 695.3910197319257
Iteration:      8,   Func. Count:    163,   Neg. LLF: 3645.434987720312
Iteration:      9,   Func. Count:    186,   Neg. LLF: 9069.436377476279
Iteration:     10,   Func. Count:    206,   Neg. LLF: 8808.240924734211
Iteration:     11,   Func. Count:    226,   Neg. LLF: 4651.991078155494
Iteration:     12,   Func. Count:    246,   Neg. LLF: 4521.40034914665
Iteration:     13,   Func. Count:    266,   Neg. LLF: 5303.13721937419
Iteration:     14,   Func. Count:    286,   Neg. LLF: 4747.990715

...

C:\ProgramData\Anaconda3\lib\site-packages\arch\__future__\_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.t

Iteration:      1,   Func. Count:     20,   Neg. LLF: 604.1073298735655
Iteration:      2,   Func. Count:     42,   Neg. LLF: 381.8584647711277
Iteration:      3,   Func. Count:     64,   Neg. LLF: 46658790.6122835
Iteration:      4,   Func. Count:     84,   Neg. LLF: 278.61486469188713
Iteration:      5,   Func. Count:    104,   Neg. LLF: 36.967982106604936
Iteration:      6,   Func. Count:    124,   Neg. LLF: 34.829458201477465
Iteration:      7,   Func. Count:    145,   Neg. LLF: 238.42985021651256
Iteration:      8,   Func. Count:    165,   Neg. LLF: 67.39985537848392
Iteration:      9,   Func. Count:    185,   Neg. LLF: 97.30869759577313
Iteration:     10,   Func. Count:    205,   Neg. LLF: 26.948285037781318
Iteration:     11,   Func. Count:    225,   Neg. LLF: 23.161166533899028
Iteration:     12,   Func. Count:    245,   Neg. LLF: 27.802463091738833
Iteration:     13,   Func. Count:    265,   Neg. LLF: 25.533598919498285
Iteration:     14,   Func. Count:    285,   Neg. LLF: 21.

C:\ProgramData\Anaconda3\lib\site-packages\arch\__future__\_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.t

Iteration:      1,   Func. Count:     20,   Neg. LLF: 626.9347962845792
Iteration:      2,   Func. Count:     42,   Neg. LLF: 347.3092511485651
Iteration:      3,   Func. Count:     64,   Neg. LLF: 47439321.636592835
Iteration:      4,   Func. Count:     84,   Neg. LLF: 276.0172251217534
Iteration:      5,   Func. Count:    104,   Neg. LLF: 36.377215196311674
Iteration:      6,   Func. Count:    124,   Neg. LLF: 33.994908770692255
Iteration:      7,   Func. Count:    145,   Neg. LLF: 237.88397067270796
Iteration:      8,   Func. Count:    165,   Neg. LLF: 74.48131502288575
Iteration:      9,   Func. Count:    185,   Neg. LLF: 87.48908893693495
Iteration:     10,   Func. Count:    205,   Neg. LLF: 28.352964187996527
Iteration:     11,   Func. Count:    225,   Neg. LLF: 22.683483824528373
Iteration:     12,   Func. Count:    244,   Neg. LLF: 22.495458187107303
Iteration:     13,   Func. Count:    264,   Neg. LLF: 32.02307667726396
Iteration:     14,   Func. Count:    285,   Neg. LLF: 22.

C:\ProgramData\Anaconda3\lib\site-packages\arch\__future__\_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


In [4]:
print(Errors_A[3].mean())
print(Errors_B[3].mean())
print(Errors_C[3].mean())

0.5385309231969149
1.9241186924238594
0.5266580570666867


In [5]:
stock = web.get_data_yahoo("NVDA",
start = "2018-01-01",
end = "2021-11-17")
y=stock['Adj Close']
Y_t=y.T
Y_t=Y_t.iloc[1:978]
Y_train=Y_t
model = pm.auto_arima(Y_train, start_p=3, start_q=3,start_d=2,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=6, max_q=6, max_d=2, # maximum p and q
                      m=1,              # frequency of series
                      d=1,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      #start_P=0, 
                      #D=1, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

print(model.summary())




#ARIMA model
Errors_000=[]
for i in range(0,12):
    Y_train=Y_t.iloc[:965+i]
    Y_test=Y_t.iloc[965+i:966+i]
    
    arima_model = ARIMA(Y_train, order=model.order)
    arima_model_fit = arima_model.fit()
    preds_arima, se, conf = arima_model_fit.forecast(len(Y_test),alpha=0.3)  
    preds = pd.Series(preds_arima, index=Y_test.index)
    
    mse = mean_squared_error(Y_test, preds)
    rmse = sqrt(mean_squared_error(Y_test, preds))
    mae = mean_absolute_error(Y_test, preds)
    mape=MAPE(Y_test, preds)
    Error_i = ([mse, rmse, mae, mape])
    Errors_000.append(Error_i)
    Errors_A=pd.DataFrame(Errors_000)



#Garch model   
Errors_001=[]
for i in range(0,12): 
    Y_train=Y_t.iloc[:965+i]
    Y_test=Y_t.iloc[965+i:966+i]

    garch = arch_model(Y_train, vol="Garch", p=6, o=2, q=6, dist="skewt")
    garch_fitted = garch.fit()
    garch_forecast = garch_fitted.forecast(horizon=1)
    garch_pred_mean = garch_forecast.mean['h.1'].iloc[-1]
    preds = pd.DataFrame([garch_pred_mean])

    mse = mean_squared_error(Y_test, preds)
    rmse = sqrt(mean_squared_error(Y_test, preds))
    mae = mean_absolute_error(Y_test, preds)
    mape=MAPE(Y_test, preds)
    Error_i = ([mse, rmse, mae, mape])
    Errors_001.append(Error_i)
    Errors_B=pd.DataFrame(Errors_001)

    
#ARIMA-GARCH model  
Errors_002=[]
for i in range(0,12):
    Y_train=Y_t.iloc[:965+i]
    Y_test=Y_t.iloc[965+i:966+i]
    
    arima_model = ARIMA(Y_train, order=(model.order))
    arima_model_fit = arima_model.fit()
    arima_residuals = arima_model_fit.resid
    predicted_mu = arima_model_fit.forecast(len(Y_test))[0]

    garch = arch_model(arima_residuals, vol="Garch", p=7, o=2, q=7, dist="gaussian")
    garch_fitted = garch.fit()
    garch_forecast = garch_fitted.forecast(horizon=1)
    garch_pred_mean = garch_forecast.mean['h.1'].iloc[-1]
    garch_pred_var = garch_forecast.variance['h.1'].iloc[-1]


    # Combine both models' output: yt = mu + et
    preds = predicted_mu + garch_pred_var
    
    mse = mean_squared_error(Y_test, preds)
    rmse = sqrt(mean_squared_error(Y_test, preds))
    mae = mean_absolute_error(Y_test, preds)
    mape=MAPE(Y_test, preds)
    Error_i = ([mse, rmse, mae, mape])
    Errors_002.append(Error_i)
    Errors_C=pd.DataFrame(Errors_002)
    
    
# print(Errors_A[3].mean())
# print(Errors_B[3].mean())
# print(Errors_C[3].mean())

Performing stepwise search to minimize aic
 ARIMA(3,1,3)(0,0,0)[0] intercept   : AIC=4901.369, Time=1.04 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=4904.086, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=4903.510, Time=0.07 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=4903.890, Time=0.09 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=4908.715, Time=0.02 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=4900.833, Time=0.79 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=4901.501, Time=0.65 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=4898.836, Time=0.50 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=4899.729, Time=0.32 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=4900.460, Time=0.24 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=4900.833, Time=0.99 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=4903.116, Time=0.21 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=4897.380, Time=0.57 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=4900.387, Time=0.19 sec
 ARIMA(4,1,1)(0,0,0)[0] intercept

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.AR

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.AR

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


...

C:\ProgramData\Anaconda3\lib\site-packages\arch\__future__\_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.t

Iteration:      1,   Func. Count:     20,   Neg. LLF: 6284.484547025842
Iteration:      2,   Func. Count:     42,   Neg. LLF: 301759183.7526516
Iteration:      3,   Func. Count:     62,   Neg. LLF: 128418.28622590835
Iteration:      4,   Func. Count:     82,   Neg. LLF: 2429.2401698596827
Iteration:      5,   Func. Count:    102,   Neg. LLF: 2705.6821847694428
Iteration:      6,   Func. Count:    123,   Neg. LLF: 6269.566392664225
Iteration:      7,   Func. Count:    144,   Neg. LLF: 2614.287196849404
Iteration:      8,   Func. Count:    164,   Neg. LLF: 2157.0963636460815
Iteration:      9,   Func. Count:    184,   Neg. LLF: 2119.8785199920144
Iteration:     10,   Func. Count:    204,   Neg. LLF: 2202.105886887737
Iteration:     11,   Func. Count:    225,   Neg. LLF: 2123.336816196833
Iteration:     12,   Func. Count:    245,   Neg. LLF: 2136.450064717532
Iteration:     13,   Func. Count:    265,   Neg. LLF: 2110.083001053416
Iteration:     14,   Func. Count:    285,   Neg. LLF: 2107.

C:\ProgramData\Anaconda3\lib\site-packages\arch\__future__\_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.t

Iteration:      1,   Func. Count:     20,   Neg. LLF: 6320.611471897029
Iteration:      2,   Func. Count:     42,   Neg. LLF: 300932569.15127003
Iteration:      3,   Func. Count:     62,   Neg. LLF: 678086.8252519264
Iteration:      4,   Func. Count:     82,   Neg. LLF: 2469.3219744677626
Iteration:      5,   Func. Count:    102,   Neg. LLF: 2662.6597972438485
Iteration:      6,   Func. Count:    123,   Neg. LLF: 2327.7421276058576
Iteration:      7,   Func. Count:    143,   Neg. LLF: 7829.600860917146
Iteration:      8,   Func. Count:    163,   Neg. LLF: 2282.6620453251285
Iteration:      9,   Func. Count:    183,   Neg. LLF: 2134.3950767601013
Iteration:     10,   Func. Count:    203,   Neg. LLF: 2154.5184302993102
Iteration:     11,   Func. Count:    223,   Neg. LLF: 2124.7247709897083
Iteration:     12,   Func. Count:    243,   Neg. LLF: 2158.030310032438
Iteration:     13,   Func. Count:    263,   Neg. LLF: 2113.3852007403484
Iteration:     14,   Func. Count:    283,   Neg. LLF: 2

C:\ProgramData\Anaconda3\lib\site-packages\arch\__future__\_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


In [6]:
print(Errors_A[3].mean())
print(Errors_B[3].mean())
print(Errors_C[3].mean())

2.4685808544834034
85.21392955773645
26.819769775787446


In [7]:
stock = web.get_data_yahoo("MSFT",
start = "2018-01-01",
end = "2021-11-17")
y=stock['Adj Close']
Y_t=y.T
Y_train=Y_t.iloc[1:978]

model = pm.auto_arima(Y_train, start_p=3, start_q=3,start_d=2,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=6, max_q=6, max_d=2, # maximum p and q
                      m=1,              # frequency of series
                      d=1,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      #start_P=0, 
                      #D=1, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

print(model.summary())




#ARIMA model
Errors_000=[]
for i in range(0,13):
    Y_train=Y_t.iloc[:963+i]
    Y_test=Y_t.iloc[963+i:964+i]
    
    arima_model = ARIMA(Y_train, order=model.order)
    arima_model_fit = arima_model.fit()
    preds_arima, se, conf = arima_model_fit.forecast(len(Y_test),alpha=0.3)  
    preds = pd.Series(preds_arima, index=Y_test.index)
    
    mse = mean_squared_error(Y_test, preds)
    rmse = sqrt(mean_squared_error(Y_test, preds))
    mae = mean_absolute_error(Y_test, preds)
    mape=MAPE(Y_test, preds)
    Error_i = ([mse, rmse, mae, mape])
    Errors_000.append(Error_i)
    Errors_A=pd.DataFrame(Errors_000)



#Garch model   
Errors_001=[]
for i in range(0,13): 
    Y_train=Y_t.iloc[:963+i]
    Y_test=Y_t.iloc[963+i:964+i]

    garch = arch_model(Y_train, vol="Garch", p=6, o=2, q=6, dist="skewt")
    garch_fitted = garch.fit()
    garch_forecast = garch_fitted.forecast(horizon=1)
    garch_pred_mean = garch_forecast.mean['h.1'].iloc[-1]
    preds = pd.DataFrame([garch_pred_mean])

    mse = mean_squared_error(Y_test, preds)
    rmse = sqrt(mean_squared_error(Y_test, preds))
    mae = mean_absolute_error(Y_test, preds)
    mape=MAPE(Y_test, preds)
    Error_i = ([mse, rmse, mae, mape])
    Errors_001.append(Error_i)
    Errors_B=pd.DataFrame(Errors_001)

    
#ARIMA-GARCH model  
Errors_002=[]
for i in range(0,13):
    Y_train=Y_t.iloc[:963+i]
    Y_test=Y_t.iloc[963+i:964+i]
    
    arima_model = ARIMA(Y_train, order=(model.order))
    arima_model_fit = arima_model.fit()
    arima_residuals = arima_model_fit.resid
    predicted_mu = arima_model_fit.forecast(len(Y_test))[0]

    garch = arch_model(arima_residuals, vol="Garch", p=7, o=2, q=7, dist="gaussian")
    garch_fitted = garch.fit()
    garch_forecast = garch_fitted.forecast(horizon=1)
    garch_pred_mean = garch_forecast.mean['h.1'].iloc[-1]
    garch_pred_var = garch_forecast.variance['h.1'].iloc[-1]


    # Combine both models' output: yt = mu + et
    preds = predicted_mu + garch_pred_mean
    
    mse = mean_squared_error(Y_test, preds)
    rmse = sqrt(mean_squared_error(Y_test, preds))
    mae = mean_absolute_error(Y_test, preds)
    mape=MAPE(Y_test, preds)
    Error_i = ([mse, rmse, mae, mape])
    Errors_002.append(Error_i)
    Errors_C=pd.DataFrame(Errors_002)
    
    
# print(Errors_A[3].mean())
# print(Errors_B[3].mean())
# print(Errors_C[3].mean())

Performing stepwise search to minimize aic
 ARIMA(3,1,3)(0,0,0)[0] intercept   : AIC=inf, Time=1.67 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=4979.553, Time=0.03 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=4933.533, Time=0.07 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=4937.398, Time=0.09 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=4984.591, Time=0.02 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=4935.064, Time=0.11 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=4935.090, Time=0.16 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=4937.064, Time=0.17 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=4942.409, Time=0.03 sec

Best model:  ARIMA(1,1,0)(0,0,0)[0] intercept
Total fit time: 2.357 seconds
                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  977
Model:               SARIMAX(1, 1, 0)   Log Likelihood               -2463.766
Date:                Mon, 22 Nov 2021   AIC           

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
C:\ProgramData\Anaconda3\lib\site-packages\arch\univariate\base.py:753: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\arch\__future__\_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set re

Iteration:      1,   Func. Count:     20,   Neg. LLF: 154775.5830443775
Iteration:      2,   Func. Count:     40,   Neg. LLF: 4852.840967958495
Iteration:      3,   Func. Count:     59,   Neg. LLF: 5398.989567114483
Iteration:      4,   Func. Count:     79,   Neg. LLF: 43701.814223007445
Iteration:      5,   Func. Count:     99,   Neg. LLF: 160467.3037536159
Iteration:      6,   Func. Count:    119,   Neg. LLF: 244470.51519396613
Iteration:      7,   Func. Count:    139,   Neg. LLF: 250479.48030957495
Iteration:      8,   Func. Count:    160,   Neg. LLF: 243119.22718492115
Iteration:      9,   Func. Count:    181,   Neg. LLF: 72693.70818996665
Iteration:     10,   Func. Count:    201,   Neg. LLF: 246958.2216606781
Iteration:     11,   Func. Count:    223,   Neg. LLF: 274090.29772599525
Iteration:     12,   Func. Count:    245,   Neg. LLF: 246996.02094008095
Iteration:     13,   Func. Count:    267,   Neg. LLF: 184070.50793767095
Iteration:     14,   Func. Count:    288,   Neg. LLF: 474

C:\ProgramData\Anaconda3\lib\site-packages\arch\univariate\base.py:753: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\arch\__future__\_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


...

C:\ProgramData\Anaconda3\lib\site-packages\arch\__future__\_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.t

Iteration:     18,   Func. Count:    363,   Neg. LLF: 2220.192357158897
Iteration:     19,   Func. Count:    382,   Neg. LLF: 2220.1851458625474
Iteration:     20,   Func. Count:    401,   Neg. LLF: 2220.1840067754283
Iteration:     21,   Func. Count:    420,   Neg. LLF: 2220.1839205598553
Iteration:     22,   Func. Count:    439,   Neg. LLF: 2220.1838922568377
Iteration:     23,   Func. Count:    458,   Neg. LLF: 2220.1838914324844
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2220.1838914324844
            Iterations: 23
            Function evaluations: 458
            Gradient evaluations: 23
Iteration:      1,   Func. Count:     20,   Neg. LLF: 4218.850880127162
Iteration:      2,   Func. Count:     42,   Neg. LLF: 324668.1768535048
Iteration:      3,   Func. Count:     62,   Neg. LLF: 3060.048993037836
Iteration:      4,   Func. Count:     83,   Neg. LLF: 3778.159388594385
Iteration:      5,   Func. Count:    104,   Neg. LLF: 3461.67775

C:\ProgramData\Anaconda3\lib\site-packages\arch\__future__\_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.t

Iteration:     13,   Func. Count:    265,   Neg. LLF: 2225.3551719044362
Iteration:     14,   Func. Count:    285,   Neg. LLF: 2225.6170212747675
Iteration:     15,   Func. Count:    305,   Neg. LLF: 2230.34717314511
Iteration:     16,   Func. Count:    325,   Neg. LLF: 2223.129937192468
Iteration:     17,   Func. Count:    344,   Neg. LLF: 2223.077862182182
Iteration:     18,   Func. Count:    363,   Neg. LLF: 2223.0709992486118
Iteration:     19,   Func. Count:    382,   Neg. LLF: 2223.0701606158937
Iteration:     20,   Func. Count:    401,   Neg. LLF: 2223.0696190356766
Iteration:     21,   Func. Count:    420,   Neg. LLF: 2223.069512707347
Iteration:     22,   Func. Count:    439,   Neg. LLF: 2223.0695025819255
Iteration:     23,   Func. Count:    458,   Neg. LLF: 2223.069500369179
Iteration:     24,   Func. Count:    476,   Neg. LLF: 2223.069500010257
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2223.069500369179
            Iterations:

C:\ProgramData\Anaconda3\lib\site-packages\arch\__future__\_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


In [8]:
print(Errors_A[3].mean())
print(Errors_B[3].mean())
print(Errors_C[3].mean())

0.7123600008223131
57.13901097685024
0.7158056174302978


In [9]:
stock = web.get_data_yahoo("GOOGL",
start = "2018-01-01",
end = "2021-11-17")
y=stock['Adj Close']
Y_t=y.T
Y_train=Y_t.iloc[1:978]

model = pm.auto_arima(Y_train, start_p=3, start_q=3,start_d=2,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=6, max_q=6, max_d=2, # maximum p and q
                      m=1,              # frequency of series
                      d=1,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      #start_P=0, 
                      #D=1, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

print(model.summary())




#ARIMA model
Errors_000=[]
for i in range(0,13):
    Y_train=Y_t.iloc[:963+i]
    Y_test=Y_t.iloc[963+i:964+i]
    
    arima_model = ARIMA(Y_train, order=model.order)
    arima_model_fit = arima_model.fit()
    preds_arima, se, conf = arima_model_fit.forecast(len(Y_test),alpha=0.3)  
    preds = pd.Series(preds_arima, index=Y_test.index)
    
    mse = mean_squared_error(Y_test, preds)
    rmse = sqrt(mean_squared_error(Y_test, preds))
    mae = mean_absolute_error(Y_test, preds)
    mape=MAPE(Y_test, preds)
    Error_i = ([mse, rmse, mae, mape])
    Errors_000.append(Error_i)
    Errors_A=pd.DataFrame(Errors_000)



#Garch model   
Errors_001=[]
for i in range(0,13): 
    Y_train=Y_t.iloc[:963+i]
    Y_test=Y_t.iloc[963+i:964+i]

    garch = arch_model(Y_train, vol="Garch", p=6, o=2, q=6, dist="skewt")
    garch_fitted = garch.fit()
    garch_forecast = garch_fitted.forecast(horizon=1)
    garch_pred_mean = garch_forecast.mean['h.1'].iloc[-1]
    preds = pd.DataFrame([garch_pred_mean])

    mse = mean_squared_error(Y_test, preds)
    rmse = sqrt(mean_squared_error(Y_test, preds))
    mae = mean_absolute_error(Y_test, preds)
    mape=MAPE(Y_test, preds)
    Error_i = ([mse, rmse, mae, mape])
    Errors_001.append(Error_i)
    Errors_B=pd.DataFrame(Errors_001)

    
#ARIMA-GARCH model  
Errors_002=[]
for i in range(0,13):
    Y_train=Y_t.iloc[:963+i]
    Y_test=Y_t.iloc[963+i:964+i]
    
    arima_model = ARIMA(Y_train, order=(model.order))
    arima_model_fit = arima_model.fit()
    arima_residuals = arima_model_fit.resid
    predicted_mu = arima_model_fit.forecast(len(Y_test))[0]

    garch = arch_model(arima_residuals, vol="Garch", p=7, o=2, q=7, dist="gaussian")
    garch_fitted = garch.fit()
    garch_forecast = garch_fitted.forecast(horizon=1)
    garch_pred_mean = garch_forecast.mean['h.1'].iloc[-1]
    garch_pred_var = garch_forecast.variance['h.1'].iloc[-1]


    # Combine both models' output: yt = mu + et
    preds = predicted_mu + garch_pred_mean
    
    mse = mean_squared_error(Y_test, preds)
    rmse = sqrt(mean_squared_error(Y_test, preds))
    mae = mean_absolute_error(Y_test, preds)
    mape=MAPE(Y_test, preds)
    Error_i = ([mse, rmse, mae, mape])
    Errors_002.append(Error_i)
    Errors_C=pd.DataFrame(Errors_002)
    
    
# print(Errors_A[3].mean())
# print(Errors_B[3].mean())
# print(Errors_C[3].mean())

Performing stepwise search to minimize aic
 ARIMA(3,1,3)(0,0,0)[0] intercept   : AIC=9200.489, Time=1.17 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=9224.325, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=9216.687, Time=0.05 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=9216.861, Time=0.10 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=9227.140, Time=0.02 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=9212.861, Time=1.33 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=9212.889, Time=1.17 sec
 ARIMA(4,1,3)(0,0,0)[0] intercept   : AIC=inf, Time=1.57 sec
 ARIMA(3,1,4)(0,0,0)[0] intercept   : AIC=9202.691, Time=1.53 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=9220.303, Time=0.77 sec
 ARIMA(2,1,4)(0,0,0)[0] intercept   : AIC=9223.329, Time=1.26 sec
 ARIMA(4,1,2)(0,0,0)[0] intercept   : AIC=9222.197, Time=1.27 sec
 ARIMA(4,1,4)(0,0,0)[0] intercept   : AIC=9193.419, Time=1.65 sec
 ARIMA(5,1,4)(0,0,0)[0] intercept   : AIC=9195.688, Time=1.83 sec
 ARIMA(4,1,5)(0,0,0)[0] intercept   : 

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.AR

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.AR

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\arch\univariate\base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
e

...

C:\ProgramData\Anaconda3\lib\site-packages\arch\__future__\_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.t

Iteration:      1,   Func. Count:     20,   Neg. LLF: 5037.195633303527
Iteration:      2,   Func. Count:     40,   Neg. LLF: 4943.7345672053725
Iteration:      3,   Func. Count:     61,   Neg. LLF: 4669.776456716438
Iteration:      4,   Func. Count:     82,   Neg. LLF: 4537.853426911146
Iteration:      5,   Func. Count:    102,   Neg. LLF: 4530.761118002161
Iteration:      6,   Func. Count:    122,   Neg. LLF: 4546.762830282892
Iteration:      7,   Func. Count:    142,   Neg. LLF: 4532.316040249442
Iteration:      8,   Func. Count:    162,   Neg. LLF: 4510.717812723296
Iteration:      9,   Func. Count:    182,   Neg. LLF: 4493.774496884471
Iteration:     10,   Func. Count:    202,   Neg. LLF: 4488.114758729335
Iteration:     11,   Func. Count:    221,   Neg. LLF: 4489.405698139522
Iteration:     12,   Func. Count:    241,   Neg. LLF: 4490.113043678344
Iteration:     13,   Func. Count:    261,   Neg. LLF: 4487.145218654177
Iteration:     14,   Func. Count:    280,   Neg. LLF: 4487.4813

C:\ProgramData\Anaconda3\lib\site-packages\arch\__future__\_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.t

Iteration:      1,   Func. Count:     20,   Neg. LLF: 5063.344812169854
Iteration:      2,   Func. Count:     40,   Neg. LLF: 5009.52025243899
Iteration:      3,   Func. Count:     61,   Neg. LLF: 4879.453071296523
Iteration:      4,   Func. Count:     82,   Neg. LLF: 4873.450280804909
Iteration:      5,   Func. Count:    103,   Neg. LLF: 4529.017439176714
Iteration:      6,   Func. Count:    123,   Neg. LLF: 4509.77285149823
Iteration:      7,   Func. Count:    143,   Neg. LLF: 4640.471067168263
Iteration:      8,   Func. Count:    163,   Neg. LLF: 4520.891362819985
Iteration:      9,   Func. Count:    183,   Neg. LLF: 4516.682204947141
Iteration:     10,   Func. Count:    203,   Neg. LLF: 4502.696717362408
Iteration:     11,   Func. Count:    223,   Neg. LLF: 4499.32047438267
Iteration:     12,   Func. Count:    243,   Neg. LLF: 4509.470316637829
Iteration:     13,   Func. Count:    263,   Neg. LLF: 4493.431165408309
Iteration:     14,   Func. Count:    282,   Neg. LLF: 4495.28104836

C:\ProgramData\Anaconda3\lib\site-packages\arch\__future__\_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


In [10]:
print(Errors_A[3].mean())
print(Errors_B[3].mean())
print(Errors_C[3].mean())

1.062144541346719
58.010549782199256
1.060648333493007
